In [3]:
import datetime, time, os, sys, re
import json
from pprint import pprint
from time import sleep
import pandas as pd

from ../ATB.Altmetric import Altmetric
from ../ATB.Facebook import Facebook
from ../ATB.DBConnection import DBConnection
from ../ATB.CrossRef import resolve_doi

import argparse, configparser
Config = configparser.ConfigParser()
Config.read('config.cnf')

# Load config
FACEBOOK_APP_ID = Config.get('facebook', 'app_id')
FACEBOOK_APP_SECRET = Config.get('facebook', 'app_secret')
ALTMETRIC_KEY = Config.get('altmetric', 'key')

SyntaxError: invalid syntax (<ipython-input-3-e4886b7d165c>, line 7)

In [ ]:
fb_graph = Facebook(FACEBOOK_APP_ID, FACEBOOK_APP_SECRET)
altmetric = Altmetric(api_key = ALTMETRIC_KEY)

# Problems

Problems that I encountered are documented here

## 1. Facebook API returning less shares than Altmetric.com

Example DOI: [10.1186/1741-7007-10-51](10.1186/1741-7007-10-51) which resolves to this URL https://bmcbiol.biomedcentral.com/articles/10.1186/1741-7007-10-51

Detailed Altmetric results for the DOI: [Altmetric results](https://biomedcentral.altmetric.com/details/799209/facebook)

http://www.biomedcentral.com/1741-7007/10/51/

In [58]:
url = "http://bmcbiol.biomedcentral.com/articles/10.1186/1741-7007-10-51"
doi = "10.1186/1741-7007-10-51"

### Results per Altmetric API

In [53]:
alt_url = altmetric.uri(url, fetch=True)
alt_doi = altmetric.doi(doi, fetch=True)

print("DOI and ULR have same altmetric_id: {}".format(alt_url['altmetric_id'] == alt_doi['altmetric_id']))

print("FB shares: {}".format(alt_doi['counts']['facebook']['posts_count']))

DOI and ULR have same altmetric_id: True
FB shares: 38


### Results per Facebook API

In [67]:
fb_url = fb_graph.get_object(url, fields="engagement, og_object")
fb_doi = fb_graph.get_object("http://dx.doi.org/" + doi, fields="engagement, og_object")

print("DOI and ULR have same og_object_id: {}".format(fb_url['og_object']['id'] == fb_doi['og_object']['id']))

print("FB shares for URL: {}".format(fb_url['engagement']['share_count']))
print("FB shares for DOI: {}".format(fb_doi['engagement']['share_count']))

DOI and ULR have same og_object_id: False
FB shares for URL: 1
FB shares for DOI: 0


## 2. URL <-> OpenGraph object ID

Not cool behaviour...

In [99]:
url_base = "bmcbiol.biomedcentral.com/articles/10.1186/1741-7007-10-51"
doi = "10.1186/1741-7007-10-51"

url_base = "www.nature.com/news/the-future-of-dna-sequencing-1.22787"
doi = "10.1038/550179a"

urls = ['http://' + url_base,
        'http://' + url_base + '/',
        'https://' + url_base,
        'https://' + url_base + '/']

dois = ['http://dx.doi.org/' + doi,
        'https://dx.doi.org/' + doi,
        'http://doi.org/' + doi,
        'https://doi.org/' + doi]

url_results = {}
doi_results = {}

for url in urls:
    try:
        url_results[url] = fb_graph.get_object(url, fields="og_object")['og_object']['id']
    except:
        url_results[url] = None
        
for doi in dois:
    try:
        doi_results[doi] = fb_graph.get_object(doi, fields="og_object")['og_object']['id']
    except:
        doi_results[doi] = None
        
pprint(url_results)
pprint(doi_results)

{'http://www.nature.com/news/the-future-of-dna-sequencing-1.22787': '1472429859490322',
 'http://www.nature.com/news/the-future-of-dna-sequencing-1.22787/': '1446759318778508',
 'https://www.nature.com/news/the-future-of-dna-sequencing-1.22787': '1472429859490322',
 'https://www.nature.com/news/the-future-of-dna-sequencing-1.22787/': '1649534128454620'}
{'http://doi.org/10.1038/550179a': None,
 'http://dx.doi.org/10.1038/550179a': '1472429859490322',
 'https://doi.org/10.1038/550179a': None,
 'https://dx.doi.org/10.1038/550179a': None}


In [100]:
url_base = "bmcbiol.biomedcentral.com/articles/10.1186/1741-7007-10-51"
doi = "10.1186/1741-7007-10-51"

urls = ['http://' + url_base,
        'http://' + url_base + '/',
        'https://' + url_base,
        'https://' + url_base + '/']

dois = ['http://dx.doi.org/' + doi,
        'https://dx.doi.org/' + doi,
        'http://doi.org/' + doi,
        'https://doi.org/' + doi]

url_results = {}
doi_results = {}

for url in urls:
    try:
        url_results[url] = fb_graph.get_object(url, fields="og_object")['og_object']['id']
    except:
        url_results[url] = None
        
for doi in dois:
    try:
        doi_results[doi] = fb_graph.get_object(doi, fields="og_object")['og_object']['id']
    except:
        doi_results[doi] = None
        
pprint(url_results)
pprint(doi_results)

{'http://bmcbiol.biomedcentral.com/articles/10.1186/1741-7007-10-51': '1246877015357759',
 'http://bmcbiol.biomedcentral.com/articles/10.1186/1741-7007-10-51/': '2058851297473624',
 'https://bmcbiol.biomedcentral.com/articles/10.1186/1741-7007-10-51': '2058851297473624',
 'https://bmcbiol.biomedcentral.com/articles/10.1186/1741-7007-10-51/': '2058851297473624'}
{'http://doi.org/10.1186/1741-7007-10-51': '2058851297473624',
 'http://dx.doi.org/10.1186/1741-7007-10-51': '2058851297473624',
 'https://doi.org/10.1186/1741-7007-10-51': '2058851297473624',
 'https://dx.doi.org/10.1186/1741-7007-10-51': '2058851297473624'}
